In [ ]:
%run /home/jovyan/work/testing.py

✅ Bucket created!
✅ Job completed!

Files in bucket:
  - output.parquet/_SUCCESS
  - output.parquet/part-00000-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00001-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00002-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00003-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00004-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00005-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00006-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00007-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00008-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00009-c1138f74-f87f-4e02-ba73-140ad38069cf-c000.snappy.parquet
  - output.parquet/part-00010-c1138f74-f87f-4e02-

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("TestS3") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark is ready!")

✅ Spark is ready!


In [6]:
# Read back the data
df_read = spark.read.parquet("s3a://test-bucket/output.parquet")
df_read.show()

+---+-------+
| id|doubled|
+---+-------+
| 16|     32|
| 17|     34|
| 18|     36|
| 19|     38|
| 20|     40|
| 21|     42|
| 22|     44|
| 23|     46|
| 24|     48|
| 41|     82|
| 42|     84|
| 43|     86|
| 44|     88|
| 45|     90|
| 46|     92|
| 47|     94|
| 48|     96|
| 49|     98|
| 66|    132|
| 67|    134|
+---+-------+
only showing top 20 rows



### Kafka spark



In [1]:
!pip install boto3


  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/78/b6/c717416e67b1bef84be9c05f2686d353544eda2af5b43af06996a43f6481/boto3-1.39.12-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.40.0,>=1.39.12 from https://files.pythonhosted.org/packages/c8/af/3b82594c5fa26464b548da35a6affe27a44993f65c4149ed8cf8a5df8387/botocore-1.39.12-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.14.0,>=0.13.0 from https://files.pythonhosted.org/packages/6d/4f/d073e09df851cfa251ef7840007d04db3293a0482ce607d2b993926089be/s3transfer-0.13.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 1.1 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB

In [2]:

import boto3
s3 = boto3.client('s3', endpoint_url='http://localstack:4566', 
                  aws_access_key_id='test', aws_secret_access_key='test')
s3.create_bucket(Bucket='test-bucket')
print("Bucket created!")

Bucket created!


In [3]:
!pip install kafka-python


  Obtaining dependency information for kafka-python from https://files.pythonhosted.org/packages/e6/35/e8bfed5425e8fe685bd03ec3f5135ee8b88c11558baa59c0d12fbd2a20ae/kafka_python-2.2.15-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.8/309.8 kB 1.6 MB/s eta 0:00:00a 0:00:01m


In [4]:

from kafka.admin import KafkaAdminClient, NewTopic

admin = KafkaAdminClient(bootstrap_servers='kafka:29092')
topic = NewTopic(name='test-topic', num_partitions=1, replication_factor=1)
admin.create_topics([topic])
print("Topic created!")

Topic created!


In [5]:
%run /home/jovyan/work/simple_producer.py

Sent: {'id': 0, 'name': 'Message 0', 'value': 0}
Sent: {'id': 1, 'name': 'Message 1', 'value': 100}
Sent: {'id': 2, 'name': 'Message 2', 'value': 200}
Sent: {'id': 3, 'name': 'Message 3', 'value': 300}
Sent: {'id': 4, 'name': 'Message 4', 'value': 400}
Sent: {'id': 5, 'name': 'Message 5', 'value': 500}
Sent: {'id': 6, 'name': 'Message 6', 'value': 600}
Sent: {'id': 7, 'name': 'Message 7', 'value': 700}
Sent: {'id': 8, 'name': 'Message 8', 'value': 800}
Sent: {'id': 9, 'name': 'Message 9', 'value': 900}
Done sending messages!


In [ ]:
%run /home/jovyan/work/simple_spark_consumer.py

✅ Spark session created
📖 Reading from Kafka...

📊 Data from Kafka:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


💾 Writing to Delta Lake at: s3a://test-bucket/delta-table
✅ Data saved to Delta Lake!

📖 Reading back from Delta Lake:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


✅ Test completed successfully!


In [4]:
import boto3

# Configure boto3 for LocalStack
s3_client = boto3.client(
    's3',
    endpoint_url='http://localstack:4566',
    aws_access_key_id='test',
    aws_secret_access_key='test',
    region_name='us-east-1'
)

# List objects in the Delta table path
try:
    response = s3_client.list_objects_v2(Bucket='test-bucket', Prefix='delta-table/')
    if 'Contents' in response:
        print("Files in s3://test-bucket/delta-table/:")
        for obj in response['Contents']:
            print(f" - {obj['Key']} ({obj['Size']} bytes)")
    else:
        print("No files found in s3://test-bucket/delta-table/ or bucket does not exist.")
except s3_client.exceptions.NoSuchBucket:
    print("Bucket 'test-bucket' does not exist. Creating it...")
    s3_client.create_bucket(Bucket='test-bucket')
    print("Bucket created. Rerun the Spark job to write the Delta table.")
except Exception as e:
    print(f"Error: {str(e)}")

Files in s3://test-bucket/delta-table/:
 - delta-table/_delta_log/00000000000000000000.json (1248 bytes)
 - delta-table/part-00000-354a5e3f-e191-438b-bbac-aec33463ff8b-c000.snappy.parquet (1177 bytes)


In [ ]:
# Create Spark session with Delta support
spark = SparkSession.builder \
    .appName("SimpleTest") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
output_path = "s3a://test-bucket/delta-table"
print("\n📖 Reading back from Delta Lake:")
delta_df = spark.read.format("delta").load(output_path)
delta_df.show()


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create Spark session with Delta support
spark = SparkSession.builder \
    .appName("SimpleTest") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "io.delta:delta-core_2.12:2.4.0,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localstack:4566") \
    .config("spark.hadoop.fs.s3a.access.key", "test") \
    .config("spark.hadoop.fs.s3a.secret.key", "test") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark session created")

# Read from Kafka
print("📖 Reading from Kafka...")
df = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "earliest") \
    .load()

# Parse JSON and select fields
parsed_df = df.select(
    col("value").cast("string")
).select(
    get_json_object(col("value"), "$.id").alias("id"),
    get_json_object(col("value"), "$.name").alias("name"),
    get_json_object(col("value"), "$.value").alias("value")
)

# Show data
print("\n📊 Data from Kafka:")
parsed_df.show()

# Write to Delta Lake on S3
#output_path = "s3a://test-bucket/delta-table"
output_path = "s3a://test-bucket/delta-tables/my_table"
print(f"\n💾 Writing to Delta Lake at: {output_path}")
parsed_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(output_path)

print("✅ Data saved to Delta Lake!")

# Read back from Delta to verify
print("\n📖 Reading back from Delta Lake:")
delta_df = spark.read.format("delta").load(output_path)
delta_df.show()

print("\n✅ Test completed successfully!")
spark.stop()

✅ Spark session created
📖 Reading from Kafka...

📊 Data from Kafka:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


💾 Writing to Delta Lake at: s3a://test-bucket/delta-tables/my_table
✅ Data saved to Delta Lake!

📖 Reading back from Delta Lake:
+---+---------+-----+
| id|     name|value|
+---+---------+-----+
|  0|Message 0|    0|
|  1|Message 1|  100|
|  2|Message 2|  200|
|  3|Message 3|  300|
|  4|Message 4|  400|
|  5|Message 5|  500|
|  6|Message 6|  600|
|  7|Message 7|  700|
|  8|Message 8|  800|
|  9|Message 9|  900|
+---+---------+-----+


✅ Test completed successfully!
